# Drafts

This document contains cells or code snippets that I want to keep aside, but that are not used in the rest of the files anymore.

## Obtaining climate data manually from climatedata.ca

Since we're doing calibration simulation here, we don't have a particular place where our simulation take place. It's up to us to choose where the climate data comes from, and there is not a "best" way to do things.

It's surely most likely better to use climate conditions that are closer to the average of the conditions we want to simulate in other LANDIS-II simulation rather than to extremes (especially for this first step of calibration). We also want to make sure that the location we use ca be inputted in FVSon.

I propose that we use an area near the border between the boral and temperate forest, and near the center of Ontario. It will be arbitrary.

Here is a map from of the forests of Ontario from the 2016 forest report of Ontario :

![](https://files.ontario.ca/1a-forestregion-map_e_1.png)

The city of [Chapleau](https://www.openstreetmap.org/#map=12/47.8416/-83.4106) seems to be located near the limit between the boreal and temperate forest. I will use a 5x5 grid cell (the grid cells being defined in the form of [https://climatedata.ca/download/](https://climatedata.ca/download/)) to download the data. Here is what it looks like on [https://climatedata.ca/download/](https://climatedata.ca/download/) :

![](./ChapleauClimateDataDotCA_5x5GridCellsForAveraging.png)

<details>
<summary>🖼️ (Collapsible section) Click here to show what the download form on https://climatedata.ca/download/ looks like when filled.</summary>

![](Sceenshot_FormClimateCanadaDotCA_Chapleau5x5Download.png)

</details>

The resulting data is in the files located in ReferencesAndData/ClimateDataChapleauOntario

The following sections will load and edit this data to create the climate file needed for PnET.

### Climate Indices from Ouranos/Climate Canada and Their Meanings

1. **cdd**: Consecutive Dry Days. The maximum number of consecutive days with precipitation below a certain threshold (usually 1mm).
2. **cddcold_18**: Consecutive Dry Days (with temperature constraint). The maximum number of consecutive days with precipitation below a certain threshold (likely 1mm) *and* with a daily mean temperature below 18°C.
3. **dlyfrzthw_tx0_tn-1**: Daily Freeze-Thaw cycles. Counts the number of days where the maximum temperature (tx) is above 0°C and the minimum temperature (tn) is below -1°C.
4. **first_fall_frost**: Date of the first fall frost. The first day in the autumn/winter season when the temperature drops below freezing (0°C).
5. **frost_days**: Number of frost days. The number of days with a minimum temperature below 0°C.
6. **frost_free_season**: Length of the frost-free season. The number of days between the last spring frost and the first fall frost.
7. **gddgrow_0**: Growing Degree Days (base 0°C). A measure of heat accumulation, calculated by summing the daily difference between the average temperature and a base temperature (0°C) for days when the average temperature is above the base temperature.
8. **gddgrow_5**: Growing Degree Days (base 5°C). Same as above, but with a base temperature of 5°C.
9. **hddheat_18**: Heating Degree Days (base 18°C). A measure of how much heating is required for a building, calculated by summing the difference between a base temperature (18°C) and the daily average temperature for days when the average temperature is below the base temperature.
10. **ice_days**: Number of ice days. The number of days with a maximum temperature below 0°C.
11. **last_spring_frost**: Date of the last spring frost. The last day in the spring season when the temperature drops below freezing (0°C).
12. **nr_cdd**: Number of Cooling Degree Days. Not explicitly defined in the original list, but typically refers to the number of days requiring cooling.
13. **prcptot**: Total precipitation. The sum of all precipitation (rain, snow, etc.) over a given period.
14. **r10mm**: Number of heavy precipitation days. The number of days with precipitation of 10mm or more.
15. **r1mm**: Number of precipitation days. The number of days with precipitation of 1mm or more.
16. **r20mm**: Number of very heavy precipitation days. The number of days with precipitation of 20mm or more.
17. **rx1day**: Maximum 1-day precipitation. The highest amount of precipitation recorded on a single day within a given period.
18. **rx5day**: Maximum 5-day precipitation. The highest amount of precipitation recorded over a 5-day period within a given period.
19. **tg_mean**: Mean temperature. The average of the daily average temperatures.
20. **tn_mean**: Mean minimum temperature. The average of the daily minimum temperatures.
21. **tn_min**: Minimum temperature. The lowest daily minimum temperature recorded.
22. **tnlt_m15**: Number of days with minimum temperature less than -15°C.
23. **tnlt_m25**: Number of days with minimum temperature less than -25°C.
24. **tr_18**: Number of days with daily mean temperature greater than 18°C.
25. **tr_20**: Number of days with daily mean temperature greater than 20°C.
26. **tr_22**: Number of days with daily mean temperature greater than 22°C.
27. **tx_max**: Maximum temperature. The highest daily maximum temperature recorded.
28. **tx_mean**: Mean maximum temperature. The average of the daily maximum temperatures.
29. **txgt_25**: Number of days with maximum temperature greater than 25°C.
30. **txgt_27**: Number of days with maximum temperature greater than 27°C.
31. **txgt_29**: Number of days with maximum temperature greater than 29°C.
32. **txgt_30**: Number of days with maximum temperature greater than 30°C.
33. **txgt_32**: Number of days with maximum temperature greater than 32°C.


#### Meaning of file names on the server of Climate Data Canada

https://en.wikipedia.org/wiki/Health_regions_of_Canada

Some .nc files (containing data about a climate variable) are indicated as spatially averaged for different things. For example, these files [here](https://pavics.ouranos.ca/twitcher/ows/proxy/thredds/catalog/birdhouse/disk2/cccs_portal/indices/Final/CanDCS-M6/tx_max/MS/ssp126/ensemble_percentiles/catalog.html) :

```
tx_max_mon_MBCn+PCIC-Blend_historical+ssp126_1950-2100_spatialAvgCensus_percentiles.nc
tx_max_mon_MBCn+PCIC-Blend_historical+ssp126_1950-2100_spatialAvgHealth_percentiles.nc
tx_max_mon_MBCn+PCIC-Blend_historical+ssp126_1950-2100_spatialAvgWatershed_percentiles.nc
```

- Census refers to [census regions of Canada](https://en.wikipedia.org/wiki/Census_geographic_units_of_Canada)
- Health refers to [health regions of Canada](https://en.wikipedia.org/wiki/Health_regions_of_Canada)
- Watersheds refers to the [watersheds of Canada](https://en.wikipedia.org/wiki/Watersheds_of_North_America)

Instead, I've attempted to create a Generalized Additive Model (GAM) to predict PAR (downwelling shortwave radiation or rsds in CanLEADv1) from Tmin, Tmax and Precipitations. I've used a script on the clusters on compute Canada (since the data is large). See [`job_script_ClimateDataProcessing_ONLYPOLYGON_INTERACTIONS.sh`](./ReferencesAndData/job_script_ClimateDataProcessing_ONLYPOLYGON_INTERACTIONS.sh).

The script reads .nc files for Tmin, Tmax, Precipitations and PAR from one of the replicates of CanLEADv1 ([here](https://crd-data-donnees-rdc.ec.gc.ca/CDAS/products/CanLEADv1/CanRCM4-EWEMBI-MBCn/r1_r1i1p1/)) and the polygon delimitating the Chapleau area ([`ChapleauBoundariesClimate.shp`](./ReferencesAndData/ChapleauBoundariesClimate.shp)). It then gathers the data for each variables inside the polygon, upsamples them from daily to monthly, puts all simultaneous observations of the 4 variables in every cell in a dataframe, and then creates a GAM. Plots are then produced to check the residuals and predictions of the GAM for verification.

The verification plot shows obvious biases in the residuals, indicating that the model fails to capture some of the variability properly; in particular, higher values of radiation are associated with higher residuals. Still, the fit of the model looks pretty good. Sadly, I don't see any better alternatives for now : 

![](./ReferencesAndData/PAR_GAM_Validation/residuals_vs_predicted.png)

WIP : Improving these GAMs with GAMMs (mixed effects for temporal and spatial autocorrelation) and maybe Tweedie distribution rather than the one used here.
Idea : Make one python and one R script to deploy on compute Canada; more models and functions in R. Python script (based on current scripts) reads, upsamples the data, then outputs to .csv. R make the models and creates the plot. Run it through interactive session and in current folder to try several models iteratively. Might switch Python script to R to test better upsampling.

We need PAR in the right format, but also for future conditions - as it can be influenced by clouds.


- MODIS data available at https://lpdaac.usgs.gov/products/mcd18c2v062/ ; but changes every 3 hours, is taken from satellite data => might have low quality data at some points. Might need something cleaner, and maybe already expressed monthly.
- https://essd.copernicus.org/articles/14/2007/2022/ describes a data set that seems more historical and cleaner than MODIS, but also in 3hours format, from 1984-01-01 to 2018-12-31. Download is made via FTP, meaning that might be able to automatize the process.

Other datasets are referenced in the intro. Havent found the best yet.

PnET User guide : https://cida.usgs.gov/thredds/catalog.html?dataset=cida.usgs.gov/macav2metdata_m onthly_historical . But not sure if still active.
The Thredds server is not active anymore, but I was able to use the code provided to access the data :

```python
import fsspec
import xarray as xr
import s3fs
import zarr
import dask
import cftime

zarr_url = 's3://mdmf/gdp/macav2_historical_monthly.zarr/'

fs = fsspec.filesystem('s3', anon=True, endpoint_url='https://usgs.osn.mghpcc.org/')

ds = xr.open_dataset(fs.get_mapper(zarr_url), engine='zarr', 
                             backend_kwargs={'consolidated':True}, chunks={})

print(ds.summary)
```

Sadly, this data is for the contiguous USA only; so we have to look elsewhere.

we could go with a statistical approach - use existing PAR data to build a statistical model for our landscape linking PAR to over variables we have (precipitations, temperature). A quick search seems to say that such model might have errors of 3-4 MJ/m2/day, while data of PAR in Canada can range from (here for canadian prairies) :

    July (peak summer):
        Clear sky: 25–28 MJ/m²/day
        Partly cloudy: 18–22 MJ/m²/day
        Thunderstorm day: 8–12 MJ/m²/day
        
    January:
        Sunny day: 10–12 MJ/m²/day
        Snowstorm: 3–5 MJ/m²/day

So, 3-4 MJ/m2/day might be too much of an error. Or is it ? Maybe there will be much less error on local data. 